# Block Logging

<a href="https://colab.research.google.com/github/ledatelescope/bifrost/blob/master/tutorial/05_block_logging.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>

In the previous section we looked at how to put blocks and rings together to build a simple pipeline.  In this section we will look at the tools Bifrost provides to analyze pipeline performance and how to integrate those into blocks.


In [1]:
%%capture install_log
# Import bifrost, but attempt to auto-install if needed (and we're running on
# Colab). If something goes wrong, evaluate install_log.show() in a new block
# to retrieve the details.
try:
  import bifrost
except ModuleNotFoundError as exn:
  try:
    import google.colab
  except ModuleNotFoundError:
    raise exn
  !sudo apt-get -qq install exuberant-ctags libopenblas-dev software-properties-common build-essential
  !pip install -q contextlib2 pint simplejson scipy git+https://github.com/ctypesgen/ctypesgen.git
  ![ -d ~/bifrost/.git ] || git clone https://github.com/ledatelescope/bifrost ~/bifrost
  !(cd ~/bifrost && ./configure --disable-cuda && make -j all && sudo make install)
  import bifrost


Let's start with the `CopyOp` block from the previous section:

In [2]:
import os
import json
import time
import numpy
import threading

class CopyOp(object):
    def __init__(self, iring, oring, ntime_gulp=250, guarantee=True, core=-1):
        self.iring = iring
        self.oring = oring
        self.ntime_gulp = ntime_gulp
        self.guarantee = guarantee
        self.core = core
        
    def main(self):
        with self.oring.begin_writing() as oring:
            for iseq in self.iring.read(guarantee=self.guarantee):
                ihdr = json.loads(iseq.header.tostring())
                
                print("Copy: Start of new sequence:", str(ihdr))
                
                time_tag = ihdr['time_tag']
                navg     = ihdr['navg']
                nbeam    = ihdr['nbeam']
                chan0    = ihdr['chan0']
                nchan    = ihdr['nchan']
                chan_bw  = ihdr['bw'] / nchan
                npol     = ihdr['npol']
                pols     = ihdr['pols']
                pols     = pols.replace('CR', 'XY_real')
                pols     = pols.replace('CI', 'XY_imag')

                igulp_size = self.ntime_gulp*nbeam*nchan*npol*4        # float32
                ishape = (self.ntime_gulp,nbeam,nchan,npol)
                self.iring.resize(igulp_size, igulp_size*5)
                
                ogulp_size = igulp_size
                oshape = ishape
                self.oring.resize(ogulp_size)
                
                ohdr = ihdr.copy()
                ohdr_str = json.dumps(ohdr)
                
                iseq_spans = iseq.read(igulp_size)
                with oring.begin_sequence(time_tag=time_tag, header=ohdr_str) as oseq:
                    for ispan in iseq_spans:
                        if ispan.size < igulp_size:
                            continue # Ignore final gulp
                            
                        with oseq.reserve(ogulp_size) as ospan:
                           idata = ispan.data_view(numpy.float32)
                           odata = ospan.data_view(numpy.float32)    
                           odata[...] = idata

If we want to know how this block is performing we can add timing statements to it and record the values using Bifrost's `bifrost.proclog` functionality.  This provides a lightweight logger that records information to `/dev/shm/bifrost` where it can be accessed.

To add logging to this block we would update it with:

In [3]:
from bifrost.proclog import ProcLog
from bifrost.affinity import get_core, set_core

class CopyOp(object):
    def __init__(self, iring, oring, ntime_gulp=250, guarantee=True, core=-1):
        self.iring = iring
        self.oring = oring
        self.ntime_gulp = ntime_gulp
        self.guarantee = guarantee
        self.core = core
        
        # ProcLog updates:
        self.bind_proclog = ProcLog(type(self).__name__+"/bind")
        self.in_proclog   = ProcLog(type(self).__name__+"/in")
        self.out_proclog  = ProcLog(type(self).__name__+"/out")
        self.size_proclog = ProcLog(type(self).__name__+"/size")
        self.sequence_proclog = ProcLog(type(self).__name__+"/sequence0")
        self.perf_proclog = ProcLog(type(self).__name__+"/perf")
        
        self.in_proclog.update(  {'nring':1, 'ring0':self.iring.name})
        self.out_proclog.update( {'nring':1, 'ring0':self.oring.name})
        self.size_proclog.update({'nseq_per_gulp': self.ntime_gulp})
        
    def main(self):
        set_core(self.core)
        self.bind_proclog.update({'ncore': 1, 
                                  'core0': get_core(),})
        
        with self.oring.begin_writing() as oring:
            for iseq in self.iring.read(guarantee=self.guarantee):
                ihdr = json.loads(iseq.header.tostring())
                
                self.sequence_proclog.update(ihdr)
                
                print("Copy: Start of new sequence:", str(ihdr))
                
                time_tag = ihdr['time_tag']
                navg     = ihdr['navg']
                nbeam    = ihdr['nbeam']
                chan0    = ihdr['chan0']
                nchan    = ihdr['nchan']
                chan_bw  = ihdr['bw'] / nchan
                npol     = ihdr['npol']
                pols     = ihdr['pols']
                pols     = pols.replace('CR', 'XY_real')
                pols     = pols.replace('CI', 'XY_imag')

                igulp_size = self.ntime_gulp*nbeam*nchan*npol*4        # float32
                ishape = (self.ntime_gulp,nbeam,nchan,npol)
                self.iring.resize(igulp_size, igulp_size*5)
                
                ogulp_size = igulp_size
                oshape = ishape
                self.oring.resize(ogulp_size)
                
                ohdr = ihdr.copy()
                ohdr_str = json.dumps(ohdr)
                
                prev_time = time.time()
                iseq_spans = iseq.read(igulp_size)
                with oring.begin_sequence(time_tag=time_tag, header=ohdr_str) as oseq:
                    for ispan in iseq_spans:
                        if ispan.size < igulp_size:
                            continue # Ignore final gulp
                        curr_time = time.time()
                        acquire_time = curr_time - prev_time
                        prev_time = curr_time
                        
                        with oseq.reserve(ogulp_size) as ospan:
                            curr_time = time.time()
                            reserve_time = curr_time - prev_time
                            prev_time = curr_time
                            
                            idata = ispan.data_view(numpy.float32)
                            odata = ospan.data_view(numpy.float32)    
                            odata[...] = idata
                            
                        curr_time = time.time()
                        process_time = curr_time - prev_time
                        prev_time = curr_time
                        self.perf_proclog.update({'acquire_time': acquire_time, 
                                                  'reserve_time': reserve_time, 
                                                  'process_time': process_time,})

In the `__init__` method of `CopyOp` we define all of the logging monitoring points:

 * `bind_proclog` — CPU and GPU binding information
 * `in_proclog` — input ring(s), if any
 * `out_proclog` — output ring(s), if any
 * `size_proclog` — gulp size in use
 * `sequence_proclog` — metadata for the current sequence being processed
 * `perf_proclog` — block performance timing, consisting of:
 * `acquire_time` — time spent waiting to read a span
 * `reserve_time` — time spect waiting to acquire an output span for writing
 * `process_time` — time spent processing data (basically everything else)

For some of these monitoring points, like in, out, and size, the values are known when the block is created and they can be saved during initalization.  For others, like sequence and perf, they can only be determined while running inside `main`.

In addition to these explict monitoring points, Bifrost also creates monitoring points for the rings that capture their names, sizes, and space.  All of this information is recorded to `/dev/shm/bifrost` in a per-process, per-block/ring fashion.  The path would be:

    /dev/shm/bifrost/<control_PID>/<block_name>/<monitor_point>

Bifrost provides tools, like the top-like utility `like_top.py`, that can tap into this information and provide real time information about a running pipeline.  The `bifrost.proclog` module also provides methods for accessing these data in a programmatic way.